In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import logging
tf.get_logger().setLevel(logging.ERROR)

EPOCHS = 1
BATCH_SIZE = 256
INPUT_FILE_NAME = 'C:/Users/Amelia/Documents/Computer Science Spring 2025/Neural Networks/GoShortenedExampleData.txt'
WINDOW_LENGTH = 40
WINDOW_STEP = 3
BEAM_SIZE = 8
NEXT_COORDINATES = 4
MAX_LENGTH = 50

file = open(INPUT_FILE_NAME, 'r', encoding='utf-8')
text = file.read()
file.close()

# Make lowercase and remove newline and extra spaces.
#text = text.lower() 
text = text.replace('\n', ' ')
text = text.replace('  ', ' ')
text = text.replace('B', ' ')
text = text.replace('W', ' ')
text = text.replace(';', ' ')

# Encode characters as indices.
unique_chars = list(set(text))
char_to_index = dict((ch, index) for index, ch 
                     in enumerate(unique_chars))
index_to_char = dict((index, ch) for index, ch 
                     in enumerate(unique_chars))
encoding_width = len(char_to_index)

fragments = []
targets = []
for i in range(0, len(text) - WINDOW_LENGTH, WINDOW_STEP):
    fragments.append(text[i: i + WINDOW_LENGTH])
    targets.append(text[i + WINDOW_LENGTH])
# Convert to one-hot encoded training data.
X = np.zeros((len(fragments), WINDOW_LENGTH, encoding_width))
y = np.zeros((len(fragments), encoding_width))
for i, fragment in enumerate(fragments):
    for j, char in enumerate(fragment):
        X[i, j, char_to_index[char]] = 1
    target_char = targets[i]
    y[i, char_to_index[target_char]] = 1

model = Sequential()
model.add(LSTM(128, return_sequences=True,
               dropout=0.2, recurrent_dropout=0.2,
               input_shape=(None, encoding_width)))
model.add(LSTM(128, dropout=0.2,
               recurrent_dropout=0.2))
model.add(Dense(encoding_width, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam')
model.summary()
history = model.fit(X, y, validation_split=0.05,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS, verbose=2,
                    shuffle=True)

#text prediction

next_move = ' ' 
one_hots = []
for i, char in enumerate(next_move):
    x = np.zeros(encoding_width)
    x[char_to_index[char]] = 1
    one_hots.append(x)
beams = [(np.log(1.0), next_move, one_hots)]

C:\Users\Amelia\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, None, 128)           │          77,824 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 23)                  │           2,967 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 212,375 (829.59 KB)

 Trainable params: 212,375 (829.59 KB)

 Non-trainable params: 0 (0.00 B)

90/90 - 59s - 654ms/step - loss: 2.2220 - val_loss: 1.4072


In [20]:
from tensorflow.keras.models import Model, load_model
model.save('saved_model.h5')
# path_to_dir = 'C:/Users/Amelia/Documents/Computer Science Spring 2025/Neural Networks/model'
load_model = tf.keras.models.load_model('saved_model.h5')


# Predict next coordinates for game.
for i in range(NEXT_COORDINATES):
    minibatch_list = []
    # Create minibatch from one-hot encodings, and predict.
    for triple in beams:
        minibatch_list.append(triple[2])
    minibatch = np.array(minibatch_list)
    y_predict = load_model.predict(minibatch, verbose=0)
    new_beams = []
    for j, softmax_vec in enumerate(y_predict):
        triple = beams[j]
        # Create BEAM_SIZE new beams from each existing beam.
        for k in range(BEAM_SIZE):
            char_index = np.argmax(softmax_vec)
            new_prob = triple[0] + np.log(
                softmax_vec[char_index])
            new_letters = triple[1] + index_to_char[char_index]
            x = np.zeros(encoding_width)
            x[char_index] = 1
            new_one_hots = triple[2].copy()
            new_one_hots.append(x)
            new_beams.append((new_prob, new_letters,
                              new_one_hots))
            softmax_vec[char_index] = 0
    # Prune tree to only keep BEAM_SIZE most probable beams.
    new_beams.sort(key=lambda tup: tup[0], reverse=True)
    beams = new_beams[0:BEAM_SIZE]
for item in beams:
    print(item[1])
 

 []] 
 [[]]
 [[] 
  [[]
  []]
 []  
 ] [[
 ] []
